# Draft notebook for setting up rOM3

I'll re-add more of the tutorial stuff later. Keeping it brief and functional for now

In [ ]:
import intake
import xarray as xr
import os
import subprocess
import matplotlib.pyplot as plt
from pathlib import Path
from dask.distributed import Client

#! ADD DEV VERSIONS OF RMOM6
import sys
sys.path.insert(0, '/g/data/ol01/rOM3-dev/regional-mom6')
sys.path.insert(0, '/g/data/ol01/rOM3-dev/mom6_bathy')
#!
import regional_mom6 as rmom6
print("using regional-mom6 version " + rmom6.__version__)
client = Client(threads_per_worker = 1)
client

## Step 1: Choose our domain, define workspace paths

To make sure that things are working we recommend starting with the default example defined below.

If this runs ok, then we can change to a domain of our choice and hopefully it runs ok too! If not, check the [README](https://github.com/COSIMA/regional-mom6/blob/main/README.md) and [documentation](https://regional-mom6.readthedocs.io/) for troubleshooting tips.

We can log in and use [Copernicus GUI](https://data.marine.copernicus.eu/product/GLOBAL_MULTIYEAR_PHY_001_030/download) to find the latitude-longitude ranges of the domain of choice and then paste below.

In [ ]:
expt_name = "adelaide5"
#! Override for testing
scratch = Path("/g/data/ol01/ab8992")
home = Path("/g/data/ol01/ab8992")
latitude_extent = [-37, -32.2]
longitude_extent = [131.8, 139.4]

date_range = ["2013-01-01", "2013-01-05"]

## Place where all the input files go
input_dir = f"{scratch}/rom3-inputdirs/{expt_name}/"

## Directory where we'll be running the experiment from
run_dir = f"{home}/rom3-rundirs/{expt_name}/"

## Directory where ocean model cut-outs go before processing
tmp_dir = f"{scratch}/tmp/{expt_name}"

## if directories don't exist, create them
for path in (run_dir, tmp_dir, input_dir):
    os.makedirs(str(path), exist_ok=True)

In [ ]:
expt = rmom6.experiment(
    longitude_extent = longitude_extent,
    latitude_extent = latitude_extent,
    date_range = [date_range[0]+" 00:00:00", date_range[1]+" 00:00:00"],
    resolution = 0.05,
    number_vertical_layers = 75,
    layer_thickness_ratio = 10,
    depth = 4500,
    tidal_constituents=[], # Uncomment to turn tides on! This can make the domain unstable though if you've chosen poor boundaries, so good to test first with no tides.
    mom_run_dir = run_dir,
    mom_input_dir = input_dir,
    boundaries=["north", "south", "east", "west"]
)

## 2: Fetch raw initial conditions from ACCESS-OM2
This is just inherited from existing COSIMA example

In [ ]:
buffer = 0.2 # degrees; buffer around the regional domain used for interpolation -- try 2x the input's resolution
catalog = intake.cat.access_nri
print('Open dataset...')
experiment = catalog["01deg_jra55v150_iaf_cycle1"]

ds_dict = experiment.search(
    variable=["u", "v", "salt", "temp", "eta_t"],
    frequency='1day',
    start_date=date_range[0]+", 00:00:00",
    
).to_dataset_dict()

access_om2_input = xr.merge(ds_dict.values(), join="inner")

access_om2_input = access_om2_input.sel(
        yu_ocean = slice(latitude_extent[0] - buffer, latitude_extent[1] + buffer),
        yt_ocean = slice(latitude_extent[0] - buffer, latitude_extent[1] + buffer))

access_om2_input = access_om2_input.sel(time=slice(date_range[0], date_range[1]))


print('Initial condition...')

## Cut out initial condition and save to netCDF
ic = access_om2_input.isel(time = 0)

## `longitude_slicer` handles seams in longitude and different grid and ensures that the output matches our 'longitude_extent'
ic = rmom6.longitude_slicer(ic,
                            [longitude_extent[0] - buffer, longitude_extent[1] + buffer],
                            ["xu_ocean", "xt_ocean"])
ic.drop_encoding().to_netcdf(tmp_dir + "/ic_unprocessed.nc")

## Cut out East & West boundary conditions and save to netCDF
print('East & West boundary conditions...')

rmom6.longitude_slicer(access_om2_input,
                       [longitude_extent[1] - buffer, longitude_extent[1] + buffer],
                       ["xu_ocean", "xt_ocean"]).to_netcdf(tmp_dir + "/east_unprocessed.nc")

rmom6.longitude_slicer(access_om2_input,
                       [longitude_extent[0] - buffer, longitude_extent[0] + buffer],
                       ["xu_ocean", "xt_ocean"]).to_netcdf(tmp_dir + "/west_unprocessed.nc")

## Cut out North & South boundary conditions and save to netCDF
print('North & South boundary conditions...')

northsouth = rmom6.longitude_slicer(access_om2_input,
                                    [longitude_extent[0] - buffer, longitude_extent[1] + buffer],
                                    ["xu_ocean", "xt_ocean"])

northsouth.sel(
    yu_ocean = slice(latitude_extent[1] - buffer, latitude_extent[1] + buffer),
    yt_ocean = slice(latitude_extent[1] - buffer, latitude_extent[1] + buffer)
).to_netcdf(tmp_dir + "/north_unprocessed.nc")

northsouth.sel(
    yu_ocean = slice(latitude_extent[0] - buffer, latitude_extent[0] + buffer),
    yt_ocean = slice(latitude_extent[0] - buffer, latitude_extent[0] + buffer)
).to_netcdf(tmp_dir + "/south_unprocessed.nc")

print('Finished cutting out the initial and boundary conditions.')

## 3. Set up bathymetry & forcing files

In [ ]:
expt.setup_bathymetry(
    bathymetry_path='/g/data/ik11/inputs/GEBCO_2022/GEBCO_2022.nc',
    longitude_coordinate_name='lon',
    latitude_coordinate_name='lat',
    vertical_coordinate_name='elevation',
    )
expt.bathymetry.depth.where(expt.bathymetry.depth != 0).plot() #TODO add 0 as fill value in rmom6 so that it plots nicely without this ugly step

In [ ]:
ocean_varnames = {"time": "time",
                  "yh": "yt_ocean",
                  "xh": "xt_ocean",
                  "xq": "xu_ocean",
                  "yq": "yu_ocean",
                  "zl": "st_ocean",
                  "eta": "eta_t",
                  "u": "u",
                  "v": "v",
                  "tracers": {"salt": "salt", "temp": "temp"}
                  }

# Set up the initial condition.
expt.setup_initial_condition(
    tmp_dir + "/ic_unprocessed.nc", # directory where the unprocessed initial condition is stored, as defined earlier
    ocean_varnames,
    arakawa_grid = "B"
    )

# Set up the four boundary conditions.
expt.setup_ocean_state_boundaries(
    Path(tmp_dir),
    ocean_varnames,
    arakawa_grid = "B"
    )

## 4: *NEW BITS* Set up the rOM3 specific stuff

Firstly, generate the esmfmesh files. Currently this isn't incorporated into the notebook, because it's pending the integration of NCAR's tools into rmom6. This is a draft PR for now

In [ ]:
#! This cell won't be necessary once mom6_bathy is incorporated into rmom6 properly
from mom6_bathy.grid import Grid
from mom6_bathy.topo import Topo
bathy_mod = xr.open_dataset(expt.mom_input_dir / "bathymetry.nc")
bathy_mod.isel(ntiles = 0).to_netcdf(expt.mom_input_dir / "bathymetry_for_ncar.nc")
grid = Grid.from_supergrid(expt.mom_input_dir / "hgrid.nc")
topo = Topo.from_topo_file(grid, expt.mom_input_dir / "bathymetry_for_ncar.nc")
topo.write_esmf_mesh(expt.mom_input_dir / "access-rom3-ESMFmesh.nc")

# Now make the nomask mesh by modifying this output. This will be put into mom6_bathy!
maskmesh = xr.open_dataset(expt.mom_input_dir / "access-rom3-ESMFmesh.nc")
maskmesh.elementMask[:] = 1
maskmesh.to_netcdf(expt.mom_input_dir / "access-rom3-nomask-ESMFmesh.nc")

Secondly, set set up the run directory. The function below has 3 steps:

1. Copy the default regional run-directory files. They come from [here](https://github.com/ashjbarnes/access-om3-configs/tree/dev-regional-generic) for now 

2. Set up the generic stuff needed for all runs, e.g. writing to MOM_override. This is now a separate function so it can still be used for non-NCI workflows

3. Finally, modify config.yaml, nuopc.runconfig, drof_in etc to finish off the rOM3 specific stuff 

In [ ]:
expt.setup_rOM3()

## Random troubleshooting stuff below - ignore


In [ ]:
def checksame(dir1,dir2):
    cwd = Path("/g/data/ol01/ab8992/")
    for i in ["datm_in","MOM_input","nuopc.runseq","config.yaml","drv_in","input.nml","MOM_override","drof_in","nuopc.runconfig"]:
        out = subprocess.run(f"diff {dir1}/{i} {dir2}/{i}",shell=True,cwd = cwd,capture_output=True).stdout.decode('ascii')
        if out != "":
            print(i,"\n",out)
            print()

checksame(Path("rom3-rundirs/rom3-tas"),Path("rom3-rundirs/adelaide3"))

In [ ]:
temp = xr.open_dataset("/g/data/ol01/ab8992/tmp/adelaide/ic_unprocessed.nc")